# Lending Club

**Описание проекта:**

Работа является тестовым заданием для зачисления на стажировку в [ШИФТ](https://team.cft.ru/events/130) (Школа информационных и финансовых технологий).

Исходные данные - это данные онлайн-заявителей [Lending Club](https://www.lendingclub.com/) в США, которые предоставляет финансовые услуги в сфере инвестирования и кредитования физических лиц. Данные уже разбиты на выборки:
- `X_test`  - контрольная/тестовая выборка,
- `X_train` - обучающая/тренировочная выборка,
- `y_train` - целевой признак для тренировочной выборки `loan_status`: **1** - кредит не возвращен, **0** - заемщик вернул кредит.

(*) целевой признак для тестовой выборки не предоставляется. Этот датасет используется для контроля выполнения задания и находится у организаторов конкурса.

Необходимо построить модель предсказания статус возврата кредита `loan_status`. Для каждого клиента из тестовой выборки `X_test.csv` составить предсказание и сохранить результат в виде датасета с колонками `index` и `loan_status`, где проставлена вероятность _не возврата кредита_. Название датасета: `answer.csv`, и должен быть упакован в zip-архив. Ниже приведёт пример содержания файла с ответами:

```
index,loan_status
1,0
2,0
3,0
5,0
7,0
9,0
10,0
17,0
```

Для оценка модели будет использоваться метрике `Gini`:

$$
Gini = 2 * RA - 1,
$$

где **RA** - это значение **ROC AUC**.

**План работы:**
1. Чтение и знакомство с данными
2. Предобработка данных
3. Анализ данных (EDA)
4. Модели предсказания
5. Вывод
___

## Чтение и знакомство с данными

In [1]:
# базовые библиотеки
import pandas as pd
import matplotlib.pyplot as plt

# спец. библиотеки
from sklearn.metrics import roc_auc_score

In [2]:
# чтение датасетов
X_test  = pd.read_csv('X_test.csv',  index_col='index', parse_dates=['earliest_cr_line', 'issue_d'])
X_train = pd.read_csv('X_train.csv', index_col='index', parse_dates=['earliest_cr_line', 'issue_d'])
y_train = pd.read_csv('y_train.csv', index_col='index')

D:\Users\Dispers\AppData\Local\Temp\ipykernel_19164\3164908412.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X_test  = pd.read_csv('X_test.csv',  index_col='index', parse_dates=['earliest_cr_line', 'issue_d'])
D:\Users\Dispers\AppData\Local\Temp\ipykernel_19164\3164908412.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X_test  = pd.read_csv('X_test.csv',  index_col='index', parse_dates=['earliest_cr_line', 'issue_d'])
D:\Users\Dispers\AppData\Local\Temp\ipykernel_19164\3164908412.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X_train = pd.read_csv('X_train.cs

In [3]:
# срез датасета
X_train.head(10)

,acc_now_delinq,addr_state,annual_inc,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,dti,earliest_cr_line,emp_length,fico_range_high,...,pub_rec,pub_rec_bankruptcies,purpose,revol_util,tax_liens,term,title,total_acc,verification_status,zip_code
index,,,,,,,,,,,,,,,,,,,,,
0,0.0,UT,80000.0,0.0,0.0,0.0,30.49,1996-10-01,4 years,689.0,...,1.0,1.0,credit_card,56.1%,0.0,36 months,Credit card refinancing,26.0,Source Verified,847xx
1,0.0,CA,82000.0,0.0,0.0,0.0,7.00,1993-03-01,10+ years,689.0,...,1.0,1.0,credit_card,88.3%,0.0,36 months,Credit card refinancing,11.0,Not Verified,900xx
2,0.0,NV,46080.0,0.0,0.0,1.0,17.32,2000-11-01,3 years,674.0,...,0.0,0.0,credit_card,18.1%,0.0,36 months,Credit card refinancing,31.0,Source Verified,895xx
3,0.0,AZ,30000.0,0.0,0.0,0.0,7.80,2010-01-01,< 1 year,694.0,...,0.0,0.0,car,33.4%,0.0,36 months,Car financing,24.0,Source Verified,853xx
4,0.0,OH,70000.0,0.0,0.0,0.0,13.36,2002-01-01,< 1 year,674.0,...,0.0,0.0,other,88.6%,0.0,60 months,Other,16.0,Not Verified,453xx
5,0.0,TX,69454.0,0.0,0.0,0.0,7.60,1993-04-01,10+ years,704.0,...,0.0,0.0,debt_consolidation,45.3%,0.0,60 months,Debt Consolitation,21.0,Verified,767xx
6,0.0,WA,78000.0,0.0,0.0,0.0,10.82,2001-04-01,5 years,664.0,...,0.0,0.0,debt_consolidation,24.1%,0.0,60 months,Debt consolidation,29.0,Source Verified,983xx
7,0.0,FL,55000.0,0.0,0.0,1.0,17.11,1995-09-01,3 years,699.0,...,1.0,1.0,debt_consolidation,55.2%,0.0,36 months,Debt consolidation,11.0,Source Verified,322xx
8,0.0,MA,60000.0,0.0,0.0,0.0,7.79,1991-11-01,10+ years,679.0,...,0.0,0.0,debt_consolidation,85.5%,0.0,36 months,Debt consolidation,7.0,Source Verified,019xx


In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1199861 entries, 0 to 1199860
Data columns (total 44 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   acc_now_delinq              1199861 non-null  float64       
 1   addr_state                  1199861 non-null  object        
 2   annual_inc                  1199861 non-null  float64       
 3   chargeoff_within_12_mths    1199824 non-null  float64       
 4   collections_12_mths_ex_med  1199824 non-null  float64       
 5   delinq_2yrs                 1199861 non-null  float64       
 6   dti                         1199861 non-null  float64       
 7   earliest_cr_line            1199861 non-null  datetime64[ns]
 8   emp_length                  1128114 non-null  object        
 9   fico_range_high             1199861 non-null  float64       
 10  fico_range_low              1199861 non-null  float64       
 11  funded_amnt                 1

In [5]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1199861 entries, 0 to 1199860
Data columns (total 1 columns):
 #   Column       Non-Null Count    Dtype
---  ------       --------------    -----
 0   loan_status  1199861 non-null  int64
dtypes: int64(1)
memory usage: 18.3 MB


In [6]:
X_test

,acc_now_delinq,addr_state,annual_inc,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,dti,earliest_cr_line,emp_length,fico_range_high,...,pub_rec,pub_rec_bankruptcies,purpose,revol_util,tax_liens,term,title,total_acc,verification_status,zip_code
index,,,,,,,,,,,,,,,,,,,,,
0,0.0,GA,46209.0,0.0,0.0,0.0,34.98,2009-04-01,8 years,669.0,...,0.0,0.0,debt_consolidation,61.7%,0.0,60 months,Debt consolidation,20.0,Verified,302xx
1,0.0,PA,43500.0,0.0,0.0,0.0,20.25,1982-06-01,5 years,704.0,...,0.0,0.0,debt_consolidation,58.6%,0.0,60 months,POFF,32.0,Not Verified,151xx
2,0.0,OH,72000.0,0.0,0.0,0.0,15.98,1997-11-01,3 years,699.0,...,0.0,0.0,credit_card,46.2%,0.0,36 months,credit crd,36.0,Verified,440xx
3,0.0,LA,120000.0,0.0,0.0,0.0,24.93,2001-07-01,10+ years,784.0,...,0.0,0.0,other,3.7%,0.0,36 months,Other,31.0,Not Verified,710xx
4,0.0,IN,55000.0,0.0,0.0,1.0,23.22,1990-09-01,10+ years,689.0,...,0.0,0.0,credit_card,68.1%,0.0,36 months,Credit card refinancing,28.0,Not Verified,465xx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590972,0.0,CA,39000.0,0.0,0.0,1.0,26.77,2007-10-01,3 years,694.0,...,0.0,0.0,credit_card,68.7%,0.0,36 months,Credit card refinancing,20.0,Source Verified,916xx
590973,0.0,IL,80000.0,0.0,0.0,1.0,22.54,1991-11-01,10+ years,674.0,...,0.0,0.0,debt_consolidation,54.5%,0.0,36 months,Debt consolidation,37.0,Verified,601xx
590974,0.0,OH,42000.0,0.0,0.0,0.0,21.60,1976-02-01,10+ years,709.0,...,0.0,0.0,debt_consolidation,67.7%,0.0,60 months,Debt consolidation,22.0,Source Verified,431xx


In [7]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 590977 entries, 0 to 590976
Data columns (total 44 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   acc_now_delinq              590977 non-null  float64       
 1   addr_state                  590977 non-null  object        
 2   annual_inc                  590977 non-null  float64       
 3   chargeoff_within_12_mths    590958 non-null  float64       
 4   collections_12_mths_ex_med  590958 non-null  float64       
 5   delinq_2yrs                 590977 non-null  float64       
 6   dti                         590977 non-null  float64       
 7   earliest_cr_line            590977 non-null  datetime64[ns]
 8   emp_length                  555347 non-null  object        
 9   fico_range_high             590977 non-null  float64       
 10  fico_range_low              590977 non-null  float64       
 11  funded_amnt                 590977 non-null 

### Описание данных:

- `acc_now_delinq` - Кол-во счетов, по которым заемщик сейчас просрочен
- `addr_state` - Штат заемщика, указанный в кредитной заявке
- `annual_inc` - Годовой доход заёмщика
- `chargeoff_within_12_mths` - Кол-во списаний за 12 месяцев
- `collections_12_mths_ex_med` - Кол-во списаний за 12 месяцев (без учета медицинских сборов)
- `delinq_2yrs` - Кол-во просроченных задолженностей (более чем на 30 дней) за последние 2 года
- `dti` - Соотношение общей суммы ежемесячных платежей заемщика к общей сумме долговых обязательств, за исключением ипотечного кредита и запрошенного кредита, на ежемесячный доход заемщика, о котором сообщается самостоятельно
- `earliest_cr_line` - Месяц, в котором заемщик открыл самую раннюю из заявленных кредитных линий
- `emp_length` - Стаж работы в годах. Возможные значения находятся в диапазоне от 0 до 10, где 0 означает менее одного года, а 10 означает десять или более лет
- `fico_range_high` - Верхняя граница кредитного рейтинга **FICO** заемщика при выдаче кредита.
- `fico_range_low` - Нижняя граница кредитного рейтинга **FICO** заемщика при выдаче кредита
- `funded_amnt` - Общая сумма обязательств по этому кредиту на данный момент времени
- `home_ownership` - Статус собственности на жилье, предоставленный заемщиком при регистрации или полученный из кредитного отчета. Возможные значения: RENT, OWN, MORTGAGE, OTHER.
- `inq_last_12m` - Кол-во кредитных запросов за последние 12 месяцев
- `installment` - Ежемесячный платеж, причитающийся заемщику, если кредит выдан.
- `int_rate` - Процентная ставка по кредиту
- `issue_d` - Месяц, в котором был выдан кредит
- `loan_amnt` - Перечисленная сумма кредита, заявленного заемщиком. Если в какой-то момент кредитный отдел уменьшит сумму кредита, то это отразится на этом значении.
- `mort_acc` - Кол-во ипотечных счетов
- `mths_since_last_delinq` - Кол-во месяцев с момента последней просрочки заемщиком.
- `mths_since_recent_bc_dlq` - Кол-во месяцев с момента последнего просроченного платежа по банковской карте
- `mths_since_recent_inq` - Кол-во месяцев с момента последнего запроса.
- `num_accts_ever_120_pd` - Кол-во счетов, просроченных на 120 и более дней
- `num_actv_bc_tl` - Кол-во активных на данный момент банковских карт
- `num_rev_accts` - Кол-во оборотных счетов
- `num_sats` - Кол-во успешно закрытых кредитных счетов
- `num_tl_120dpd_2m` - Кол-во счетов, в настоящее время просроченных на 120 дней (обновлено за последние 2 месяца)
- `num_tl_30dpd` - Кол-во счетов, в настоящее время просроченных на 30 дней (обновлено за последние 2 месяца)
- `num_tl_90g_dpd_24m` - Кол-во счетов, просроченных на 90 и более дней за последние 24 месяца
- `num_tl_op_past_12m` - Кол-во счетов, открытых за последние 12 месяцев
- `open_acc` - Кол-во открытых кредитных линий в кредитном досье заемщика.
- `open_il_24m` - Кол-во счетов в рассрочку, открытых за последние 24 месяца
- `open_rv_24m` - Кол-во возобновляемых сделок, открытых за последние 24 месяца
- `percent_bc_gt_75` - Процент всех счетов банковских карт > 75% от лимита.
- `pub_rec` - Кол-во уничижительный публичных записей
- `pub_rec_bankruptcies` - Кол-во публичных банкротств
- `purpose` - Категория, предоставляемая заемщиком для заявки на кредит
- `revol_util` - Коэффициент использования возобновляемой линии или сумма кредита, которую заемщик использует по отношению ко всему доступному возобновляемому кредиту.
- `tax_liens` - Кол-во налоговых залогов
- `term` - Кол-во платежей по кредиту. Значения указаны в месяцах и могут быть 36 или 60.
- `title` - Название кредита, предоставленное заемщиком
- `total_acc` - Общее количество кредитных линий в настоящее время в кредитном деле заемщика
- `verification_status` - Указывает: был ли доход подтвержден LendingClub, подтверждён заёмщиком, или не подтвержден вовсе.
- `zip_code` - Первые 3 цифры почтового индекса, указанные заемщиком в кредитной заявке.

**Промежуточный вывод:**

_Тренировочная выборка_ содержит 1.199.861 объектов и 44 признака-предсказателя. Присутствуют _пропуски_. 

Присутствует значительное _несоответствие данных_ и их типов. Большинство колонок имеет тип `float64` или `object` (т.е. строчные). На этапе чтения файла двум колонкам удалось задать тип данных `datetime`. Тип данных для остальных колонок будет изменён в следующем разделе. Для двух колонок: `emp_length` и `int_rate`, требуется ручная обработка данных, поскольку данные содержащиеся в них нельзя однозначно интерпретировать как числовые или строчные.

Кол-во целевых признаков совпадает с кол-ом объектов в тренировочной выборке, тип данных соответствует.

_Тестовая выборка_ содержит 590.977 объектов и такое же кол-во признаков. Имеет аналогичные дефекты: пропуски, несоответствие типов данных.

## Предобработка данных

Типы данных, пропуски, дубликаты, аномалии/выбросы.

### Тип данных

In [8]:
# просмотр уникальных значений в колонках по отдельности
for col in X_train.columns:
    print(col)
    print(X_train[col].unique(), '\n')

acc_now_delinq
[0. 1. 2. 3. 5. 4. 6.] 

addr_state
['UT' 'CA' 'NV' 'AZ' 'OH' 'TX' 'WA' 'FL' 'MA' 'MD' 'VA' 'IN' 'TN' 'GA'
 'NH' 'SC' 'NJ' 'NC' 'MN' 'AL' 'PA' 'KS' 'NM' 'NY' 'KY' 'MI' 'CO' 'OR'
 'DC' 'IL' 'AR' 'LA' 'CT' 'VT' 'WI' 'OK' 'MS' 'MO' 'WY' 'RI' 'WV' 'SD'
 'DE' 'HI' 'MT' 'AK' 'ID' 'NE' 'ND' 'ME' 'IA'] 

annual_inc
[80000.   82000.   46080.   ... 65402.   26927.   34450.89] 

chargeoff_within_12_mths
[ 0.  1.  2.  4.  3. nan  5.  6. 10.  7.  9.  8.] 

collections_12_mths_ex_med
[ 0.  1.  2.  3.  4. nan 10.  7.  6.  5. 20. 12.  9.  8.] 

delinq_2yrs
[ 0.  1.  2.  4.  3.  5.  6.  8. 10.  7.  9. 11. 12. 13. 16. 14. 15. 21.
 30. 18. 19. 17. 27. 20. 29. 22. 26. 42. 24. 25. 28. 39.] 

dti
[30.49  7.   17.32 ... 43.3  49.03 44.41] 

earliest_cr_line
<DatetimeArray>
['1996-10-01 00:00:00', '1993-03-01 00:00:00', '2000-11-01 00:00:00',
 '2010-01-01 00:00:00', '2002-01-01 00:00:00', '1993-04-01 00:00:00',
 '2001-04-01 00:00:00', '1995-09-01 00:00:00', '1991-11-01 00:00:00',
 '1996-09-01 0

In [9]:
# смена типа данных с 'float64' на 'Int64', поддерживающий пропуски
columns_to_int = ['acc_now_delinq',
                  'chargeoff_within_12_mths',
                  'collections_12_mths_ex_med',
                  'delinq_2yrs',
                  'fico_range_high',
                  'fico_range_low',
                  'funded_amnt',
                  'inq_last_12m',
                  'loan_amnt',
                  'mort_acc',
                  'mths_since_last_delinq',
                  'mths_since_recent_bc_dlq',
                  'mths_since_recent_inq',
                  'num_accts_ever_120_pd',
                  'num_actv_bc_tl',
                  'num_rev_accts',
                  'num_sats',
                  'num_tl_120dpd_2m',
                  'num_tl_30dpd',
                  'num_tl_90g_dpd_24m',
                  'num_tl_op_past_12m',
                  'open_acc',
                  'open_il_24m',
                  'open_rv_24m',
                  'pub_rec',
                  'pub_rec_bankruptcies',
                  'tax_liens',
                  'total_acc']

for col in columns_to_int:
    X_train[col] = X_train[col].astype('Int64')
    
print(X_train[columns_to_int].info())

<class 'pandas.core.frame.DataFrame'>
Index: 1199861 entries, 0 to 1199860
Data columns (total 28 columns):
 #   Column                      Non-Null Count    Dtype
---  ------                      --------------    -----
 0   acc_now_delinq              1199861 non-null  Int64
 1   chargeoff_within_12_mths    1199824 non-null  Int64
 2   collections_12_mths_ex_med  1199824 non-null  Int64
 3   delinq_2yrs                 1199861 non-null  Int64
 4   fico_range_high             1199861 non-null  Int64
 5   fico_range_low              1199861 non-null  Int64
 6   funded_amnt                 1199861 non-null  Int64
 7   inq_last_12m                632646 non-null   Int64
 8   loan_amnt                   1199861 non-null  Int64
 9   mort_acc                    1168320 non-null  Int64
 10  mths_since_last_delinq      592418 non-null   Int64
 11  mths_since_recent_bc_dlq    282665 non-null   Int64
 12  mths_since_recent_inq       1050080 non-null  Int64
 13  num_accts_ever_120_pd       1154

### Пропуски

Колонки с пропусками:
`chargeoff_within_12_mths`,
`collections_12_mths_ex_med`,
`emp_length`,
`inq_last_12m`,
`mort_acc`,
`mths_since_last_delinq`,
`mths_since_recent_bc_dlq`,
`mths_since_recent_inq`,
`num_accts_ever_120_pd`,
`num_actv_bc_tl`,
`num_rev_accts`,
`num_sats`,
`num_tl_120dpd_2m`,
`num_tl_30dpd`,
`num_tl_90g_dpd_24m`,
`num_tl_op_past_12m`,
`open_il_24m`,
`open_rv_24m `,
`percent_bc_gt_75`,
`pub_rec_bankruptcies`,
`revol_util`,
`tax_liens`,
`zip_code`.


### Аномалии

**Промежуточный вывод:**
___

## Анализ данных (EDA)

## Модели предсказаний

## Вывод

Рекомендация к источнику данных: использовать соответствующий тип данных. Это позволит уменьшить объём датасета, а так же время предобработки данных, без потери качества.